# Task 1 - ETL Scripting

In [1]:
import json
import requests
import pprint
import pandas as pd
import numpy as np
import math

## Part 1 - Mine Basic Player Information
Given a list of playerId's, we need to mine basic information that includes a player's last name, first name, position, shooting/catching hand, height, weight, and date of birth.

##### Approach
* Read in csv file containing playerId's of interest
* For each playerId, retrieve player info from NHL's public API ([example for Sean Couturier](https://statsapi.web.nhl.com/api/v1/people/8476461))

In [2]:
# read csv containing playerId's into dataframe
with open('../data/nhl_draft_2020_ids.csv') as f:
    df_ids = pd.read_csv(f)

In [3]:
player_info = [] # list tot store player information

for playerId in df_ids['nhlPlayerID']:
    # pull data from api
    r = requests.get(url=f'https://statsapi.web.nhl.com/api/v1/people/{playerId}')
    d = r.json()
    
    # store in list
    player_info.append({
        'playerId': d['people'][0]['id'],
        'lastName': d['people'][0]['lastName'],
        'firstName': d['people'][0]['firstName'],
        'position': d['people'][0]['primaryPosition']['abbreviation'],
        'shootsCatches': d['people'][0]['shootsCatches'],
        'height': d['people'][0]['height'],
        'weight': d['people'][0]['weight'],
        'birthDate': d['people'][0]['birthDate'],
    })

# convert to dataframe
df_players = pd.DataFrame(player_info)

# convert height to inches
df_players[['feet', 'inches']] = df_players['height'].str.split(' ', expand=True)
df_players['feet'] = df_players['feet'].str.extract('(\d+)', expand=True)
df_players['inches'] = df_players['inches'].str.extract('(\d+)', expand=True)
df_players['height'] = df_players['feet'].astype(int) * 12 + df_players['inches'].astype(int)
df_players = df_players.drop(['feet', 'inches'], axis=1)

In [4]:
df_players.head(10)

,playerId,lastName,firstName,position,shootsCatches,height,weight,birthDate
0,8482466,Jamsen,Aatu,RW,L,73,154,2002-07-22
1,8482482,Wilsby,Adam,D,L,72,183,2000-08-07
2,8482197,Raska,Adam,RW,R,70,178,2001-09-25
3,8482503,Lyckasen,Albert,D,R,71,187,2001-07-29
4,8482494,Sundsvik,Albin,C,L,74,187,2001-04-27
5,8482523,Young,Alex,C,R,71,170,2001-03-22
6,8482182,Cotton,Alex,D,R,74,190,2001-05-12
7,8482154,Jefferies,Alex,LW,R,72,195,2001-11-08
8,8482155,Laferriere,Alex,RW,R,72,173,2001-10-28
9,8482212,Pashin,Alexander,RW,L,68,154,2002-07-28


## Part 2 - Scrape
Now, we need to acquire the following information about each player: 
* Round
* Overall Pick Number
* Drafting Team
* Team they were drafted from.

##### Approach
* Build webscraper to scrape the data from the table produced by [Hockey-Ref](https://www.hockey-reference.com/draft/NHL_2020_entry.html)
* Merge with dataframe containing basic player information
* Do basic data cleaning
* Save data to csv file

In [5]:
def get_draft_picks(soup):
    draft_picks = []
    draft_round = 1
    for row in soup.find('tbody').find_all('tr'):
        draft_overall = row.find('th').text # extract draft overall
        data = row.find_all('td')           # extract other relevant info, append to list
        if len(data) > 0:
            draft_picks.append({
                'draftRound': draft_round,
                'draftOverall': draft_overall,
                'draftingTeam': data[0].text,
                'fullName': data[1].text,
                'amateurTeam': data[6].text
            })
        else:
            draft_round += 0.5
            
    return draft_picks

In [6]:
#### Building Webscraper ####
from bs4 import BeautifulSoup
from urllib.request import urlopen
import unidecode

# fix ssl certificate (needed for MacOS sometimes)
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# get html, convert to bs4 object
url = 'https://www.hockey-reference.com/draft/NHL_2020_entry.html'
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, 'html.parser')

# scrape draft picks
draft_picks = get_draft_picks(soup)

Now we need to merge with the original dataframe. There is no common playerId, so we will merge on lastName and firstName instead.

*Note - this type of merge would fail on larger datasets where multiple players have a common lastName and firstName*

##### Challenges
* Names have diacritics that appear in nhl data but not in hockey-ref data and vice versa. These need to be dealt with.
* Some players have mismatching first and last names between the nhl data and the hockey-ref data. This leads to exploring other keys, such as birthDate, to merge the datasets on.
* Some players have completely incorrect names inputted in hockey-ref. For example, this player's [nhl profile](https://www.nhl.com/player/yauheni-aksiantsiuk-8481863) and [hockey-ref](https://www.hockey-reference.com/players/o/oksenye01.html) profile have the same draft and nationality info, but two completely different names.
* A combination of keys can be used to capture all edge cases. However, it is much easier to replace the errored records with the correct name.

In [7]:
#### Merging Datasets ####

# manually correct errors in hockey-ref data
for row in draft_picks:
    if row['fullName'] == 'John-Jason Peterka':
        row['fullName'] = 'JJ Peterka'   
    elif row['fullName'] == 'Nicolas Daws':
        row['fullName'] = 'Nico Daws'   
    elif row['fullName'] == 'Yevgeni Oksentyuk':
        row['fullName'] = 'Yauheni Aksiantsiuk'
        
# convert to dataframe, convert draftRound and draftOverall to int format
df_picks = pd.DataFrame(draft_picks)
df_picks['draftRound'] = df_picks['draftRound'].astype(int)
df_picks['draftOverall'] = df_picks['draftOverall'].astype(int)

# create a fullName column in df_players
df_players['fullName'] = df_players[['firstName', 'lastName']].agg(' '.join, axis=1)

# normalize names to remove diactrics
df_picks['fullName'] = [unidecode.unidecode(name) for name in df_picks['fullName']]
df_players['fullName'] = [unidecode.unidecode(name) for name in df_players['fullName']]

# merge on fullName
df_players = df_players.merge(df_picks, on='fullName')
df_players = df_players.drop('fullName', axis=1)

In [8]:
df_players[df_players['draftRound']==1].sort_values('draftOverall')

,playerId,lastName,firstName,position,shootsCatches,height,weight,birthDate,draftRound,draftOverall,draftingTeam,amateurTeam
14,8482109,Lafrenière,Alexis,LW,L,74,196,2001-10-11,1,1,New York Rangers,Rimouski Oceanic (QMJHL)
165,8482124,Byfield,Quinton,C,L,77,220,2002-08-19,1,2,Los Angeles Kings,Sudbury Wolves (OHL)
190,8482116,Stützle,Tim,LW,L,72,190,2002-01-15,1,3,Ottawa Senators,Mannheim (GERMANY)
127,8482078,Raymond,Lucas,LW,R,71,182,2002-03-28,1,4,Detroit Red Wings,Frolunda (Sweden)
95,8482105,Sanderson,Jake,D,L,74,185,2002-07-08,1,5,Ottawa Senators,USA U-18 (U.S. NTDP)
100,8482142,Drysdale,Jamie,D,R,71,183,2002-04-08,1,6,Anaheim Ducks,Erie Otters (OHL)
11,8482125,Holtz,Alexander,RW,R,72,195,2002-01-23,1,7,New Jersey Devils,Djurgarden (Sweden)
88,8482097,Quinn,Jack,RW,R,72,185,2001-09-19,1,8,Buffalo Sabres,Ottawa 67's (OHL)
138,8482079,Rossi,Marco,C,L,69,182,2001-09-23,1,9,Minnesota Wild,Ottawa 67's (OHL)
42,8482149,Perfetti,Cole,C,L,71,177,2002-01-01,1,10,Winnipeg Jets,Saginaw Spirit (OHL)


In [9]:
#### Save to CSV ####
df_players.to_csv('../data/nhl_draft_2020_detailed.csv', index=False)